In [2]:
#SCRAPE MULTIPLE FINANCIAL PARAMETERS FROM SINGLE PAGE / V2 / Faster scrolling using drag&drop apprach
#https://stackoverflow.com/questions/62119348/how-to-scroll-horizontally-using-selenium-chromedriver-in-python

import pandas as pd
import matplotlib.pyplot as plt
import time
import requests
import json
from html.parser import HTMLParser
import sqlite3


In [35]:
#MAIN SCRAPPER

database_file = 'company_database.db'

#HTML parser that help to parse html-strings
#https://www.educative.io/answers/what-is-the-html-parser-in-python
class Parser(HTMLParser):
  def handle_data(self, data):
    self.output = data
parser = Parser()


class ScrapeMacrotrend():

    def __init__(self, ticker_search, scrape_again=False):
        
        url = self.get_company_page_url(ticker_search) #find url for the respective company based on ticker-name
        # url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/income-statement?freq=Q'


        #scrape and store company quarter data if not available in database. If data already exists in database, return it from there.
        if self.data_already_in_database(name= (ticker_search + '_financial_statements_quarter'))==False or scrape_again==True:
            self.financial_statements_quarter = self.scrape_the_data(url + '/income-statement?freq=Q')
            self.store_data_in_database(data=self.financial_statements_quarter, name= (ticker_search + '_financial_statements_quarter'))
        else:
            self.financial_statements_quarter = self.get_from_database(name= (ticker_search + '_financial_statements_quarter'))

        if self.data_already_in_database(name= (ticker_search + '_balance_sheet_quarter'))==False or scrape_again==True:
            self.balance_sheet_quarter = self.scrape_the_data(url + '/balance-sheet?freq=Q')
            self.store_data_in_database(data=self.balance_sheet_quarter, name= (ticker_search + '_balance_sheet_quarter'))
        else:
            self.balance_sheet_quarter = self.get_from_database(name= (ticker_search + '_balance_sheet_quarter'))

        if self.data_already_in_database(name= (ticker_search + '_cash_flow_quarter'))==False or scrape_again==True:
            self.cash_flow_quarter = self.scrape_the_data(url + '/cash-flow-statement?freq=Q')
            self.store_data_in_database(data=self.cash_flow_quarter, name= (ticker_search + '_cash_flow_quarter'))
        else:
            self.cash_flow_quarter = self.get_from_database(name= (ticker_search + '_cash_flow_quarter'))


        if self.data_already_in_database(name= (ticker_search + '_financial_ratios_quarter'))==False or scrape_again==True:
            self.financial_ratios_quarter = self.scrape_the_data(url + '/financial-ratios?freq=Q')
            self.store_data_in_database(data=self.financial_ratios_quarter, name= (ticker_search + '_financial_ratios_quarter'))
        else:
            self.financial_ratios_quarter = self.get_from_database(name= (ticker_search + '_financial_ratios_quarter'))
        
        #scrape company anual data
        if self.data_already_in_database(name= (ticker_search + '_financial_statements_annual'))==False or scrape_again==True:
            self.financial_statements_annual = self.scrape_the_data(url + '/income-statement?freq=A')
            self.store_data_in_database(data=self.financial_statements_annual, name= (ticker_search + '_financial_statements_annual'))
        else:
            self.financial_statements_annual = self.get_from_database(name= (ticker_search + '_financial_statements_annual'))

        if self.data_already_in_database(name= (ticker_search + '_balance_sheet_annual'))==False or scrape_again==True:
            self.balance_sheet_annual = self.scrape_the_data(url + '/balance-sheet?freq=A')
            self.store_data_in_database(data=self.balance_sheet_annual, name= (ticker_search + '_balance_sheet_annual'))
        else:
            self.balance_sheet_annual = self.get_from_database(name= (ticker_search + '_balance_sheet_annual'))

        if self.data_already_in_database(name= (ticker_search + '_cash_flow_annual'))==False or scrape_again==True:
            self.cash_flow_annual = self.scrape_the_data(url + '/cash-flow-statement?freq=A')
            self.store_data_in_database(data=self.cash_flow_annual, name= (ticker_search + '_cash_flow_annual'))
        else:
            self.cash_flow_annual = self.get_from_database(name= (ticker_search + '_cash_flow_annual'))

        if self.data_already_in_database(name= (ticker_search + '_financial_ratios_annual'))==False or scrape_again==True:
            self.financial_ratios_annual = self.scrape_the_data(url + '/financial-ratios?freq=A')
            self.store_data_in_database(data=self.financial_ratios_annual, name= (ticker_search + '_financial_ratios_annual'))
        else:
            self.financial_ratios_annual = self.get_from_database(name= (ticker_search + '_financial_ratios_annual'))

    def get_company_page_url_old(self, ticker_search):
        #get ticker data
        tickers_url = 'https://www.macrotrends.net/assets/php/ticker_search_list.php?_=1664947632720'
        data_tickers = requests.get(tickers_url)
        data_tickers = data_tickers.text
        data_tickers_json = json.loads(data_tickers)

        #search for the company-url
        for item in data_tickers_json:
            # print(item['s'])
            if ticker_search in item['s']: #example: AAPL/apple
                company_page_url = item['s']
                # print("company-url={}".format(company_page_url))

        return 'https://www.macrotrends.net/stocks/charts/' + company_page_url

    def get_company_page_url(self, ticker_search):

        self.tickers_data = self.get_from_database('tickers_data') #get tickers-data from database
        if self.tickers_data is None: #if tickers-data not found in database
            print(f'tickers-data not found in {database_file}')
            self.tickers_data = self.download_tickers()
            self.store_data_in_database(self.tickers_data, "tickers_data")

        else: #if tickers-data exists in database
            print(f'tickers-data found in {database_file}')
            # print(self.tickers_data)

        output = self.tickers_data.loc[self.tickers_data['ticker_name'] == ticker_search]
        ticker_name = output['ticker_name'].values[0] #get ticker-name
        company_name = output['company_name'].values[0] #get company-name
        # print(ticker_name, company_name)

        return'https://www.macrotrends.net/stocks/charts/' + ticker_name + '/' + company_name

    def download_tickers(self):
        ticker_name = [] #store ticker-names f.e. 'APPL'
        company_name = [] #store company-names f.e. 'apple'
        tickers_url = 'https://www.macrotrends.net/assets/php/ticker_search_list.php?_=1664947632720' #donwnload ticker-page
        
        data_tickers = requests.get(tickers_url) #download tickers-page
        data_tickers = data_tickers.text #convert it to text
        data_tickers_json = json.loads(data_tickers) #convert the text to json-format

        for item in data_tickers_json: #loop over each item in json-format
            ticker_name_temp = item['s'].split('/')[0] #get ticker-name from current item
            company_name_temp = item['s'].split('/')[1] #get company-name from current item

            ticker_name.append(ticker_name_temp) #append ticker-name to list
            company_name.append(company_name_temp) #append company-name to list

        data = {'ticker_name':ticker_name, 'company_name':company_name} #store in dictionary
        output = pd.DataFrame(data)
        return output

    def scrape_the_data(self, url):

        page = requests.get(url)
        page_lines = page.text.splitlines()
        for line in page_lines:
            if 'var originalData =' in line: #the line where all the data is storred
                data = line #store the information in a variable
                print(f'line={url, line[0:100]}')

        data = data[20:-1] # remove the the unneeded informatino from the line
        data_json = json.loads(data) #convert the line to a json-object

        #the following vars are needed when looping through the json_
        data_index = [] #collect the data-names
        data_column = [] #collect the data-dates
        data_values = [] #collect the data-values

        #loop over 
        for item in data_json:

            #reset temp-vars for the next loop
            data_values_temp = [] #store data-values for current loop
            data_column_temp = [] #store data-dates for current loop

            for key in item.items() :
                # print(key[0], key[1])

                if (key[0] != 'field_name') and (key[0] != 'popup_icon'): # dates and values
                    # print(key[0], key[1])
                    data_column_temp.append(key[0]) #dates
                    data_values_temp.append(key[1]) #values

                elif key[0] == 'field_name': #data-names
                    # print(key[1])
                    parser.feed(key[1])
                    # print(parser.output)
                    data_index.append(parser.output) # xxx = "<a href='/stocks/charts/TSLA/tesla/cost-goods-sold'>Cost Of Goods Sold</a>"

                elif key[0]== 'popup_icon': #data-graph link (not needed)
                    continue
                    print(key[1])

            #add temp-vars to permanent ones
            data_values.append(data_values_temp)
            data_column.append(data_column_temp)


        # pd.DataFrame(data=data_values, index=data_index, columns=data_column, dtype=None, copy=None)
        company_data = pd.DataFrame(data=data_values, index=data_index, columns=data_column[0])
        company_data.apply(self.fixData)

        return company_data

    #apply-function that goes over all dataframe-elements and converts them to numeric value if possible
    def fixData(sekf, input_data):
        
        for i in range(len(input_data)):
        
            try:
                input_data[i] = pd.to_numeric(input_data[i])
                # print(type(input_data[i]), "converted to numeric")

            except:
                # pass
                print(input_data[i], "Can't convert to numeric")
            
        return input_data
        
    def store_data_in_database(self, data, name):
        print(f'Storring {name} in {database_file}')
        conn = sqlite3.connect(database_file) #connect to sqlite3-database
        data.to_sql(name=name, con=conn, if_exists='replace',  index=False) #store dataframe to sqlite-database
        conn.close()

    def data_already_in_database(self, name):

        output = False
        conn = sqlite3.connect('company_database.db')
        c = conn.cursor()
                    
        #get the count of tables with the name
        c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='{}' '''.format(name))

        #if the count is 1, then table exists
        if c.fetchone()[0]==1:
            # print('Table exists.', name)
            output = True
			
        #commit the changes to db			
        conn.commit()
        #close the connection
        conn.close()

        # print(name, output)
        return output

    def get_from_database(self, name):
        conn = sqlite3.connect(database_file)

        try: #get the data from database and put it in pandas-dataframe
            output = pd.read_sql_query(f"SELECT * from {name}", conn) 
        except : #if data not available, return None
            print(f'ERROR: {name}, not found in {database_file}')
            conn.close()
            return None

        if name != 'tickers_data': #if dataframe is not tickers-data(ticker-names) but company-data(financial-statement, balance sheet etc)
            output = output.set_index('index', inplace=False) #set the index-column as dataframe-index, needed because otherwise the dataframe index won't be parameter names but integers
        conn.close()
        return output


class DatabaseOperations():

    def drop_table(input_data):
        #Connecting to sqlite

        conn = sqlite3.connect(database_file)

        #Creating a cursor object using the cursor() method
        cursor = conn.cursor()

        #Doping EMPLOYEE table if already exists
        cursor.execute(f"DROP TABLE {input_data}")
        print(f"Table {input_data} dropped... ")

        #Commit your changes in the database
        conn.commit()

        #Closing the connection
        conn.close()

    def get_database_tables():

        con = sqlite3.connect(database_file)
        cursor = con.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        print(cursor.fetchall())

    def drop_ticker_name_tables(ticker_name):
        tables_names = ['_financial_statements_quarter', '_balance_sheet_quarter', '_cash_flow_quarter', '_financial_ratios_quarter',
                        '_financial_statements_annual', '_balance_sheet_annual', '_cash_flow_annual', '_financial_ratios_annual']

        for item in tables_names:
            # print(ticker_name+item)
            # DatabaseOperations.drop_table(ticker_name + item)
            __class__.drop_table(ticker_name + item)
            
    # company_data.dtypes
    # company_data.columns
    # company_data.index

    # company_data.loc['Revenue']
    # company_data.loc['EPS - Earnings Per Share']

    # company_data.columns = pd.to_datetime(company_data.columns) #change columns data-type to datetime.
    # company_data.columns

# pepsi_data = ScrapeMacrotrend(ticker_search='PEP', scrape_again=False)

# DatabaseOperations.drop_table(input_data='tickers_data') #remove table from database
# DatabaseOperations.get_database_tables()
DatabaseOperations.drop_ticker_name_tables(ticker_name='PEP')



OperationalError: no such table: PEP_financial_statements_quarter

In [ ]:
apple_data = ScrapeMacrotrend('AAPL')
tesla_data = ScrapeMacrotrend('TSLA')
microsoft_data = ScrapeMacrotrend('MSFT')
amazon_data = ScrapeMacrotrend('AMZN')
# google_data = ScrapeMacrotrend('GOOG')

# apple_data1 = ScrapeMacrotrend('AAPL')
# tesla_data1 = ScrapeMacrotrend('TSLA')
# microsoft_data1 = ScrapeMacrotrend('MSFT')
# amazon_data1 = ScrapeMacrotrend(ticker_search='AMZN')

#tesla_data.financial_statements_annual
# print(amazon_data.financial_statements_quarter.index.to_list())
# print(amazon_data1.financial_statements_quarter.index.to_list())

# amazon_data.financial_statements_quarter.columns

# amazon_data2 = amazon_data1.financial_statements_quarter.copy()
# print(amazon_data2.columns)
# amazon_data2.drop('index', axis=1, inplace=True) 
# amazon_data2.set_index('index', inplace=True)
# print(amazon_data2.columns)

# amazon_data.financial_statements_quarter
# amazon_data1.financial_statements_quarter
# amazon_data2

# amazon_data3 = ScrapeMacrotrend(ticker_search='AMZN')
# amazon_data3.financial_statements_quarter


In [ ]:
#plotting the data
plt.figure(figsize=(20,8)) #set figure size

# tesla_data.financial_ratios_quarter.loc['Current Ratio'].plot()
# apple_data.financial_ratios_quarter.loc['Current Ratio'].plot()

print(tesla_data.financial_statements_annual.index[0])
param_name = tesla_data.financial_statements_annual.index[4] #get parameter name from dataframe columns

tesla_data.financial_statements_annual.loc[param_name].plot()
apple_data.financial_statements_annual.loc[param_name].plot()
microsoft_data.financial_statements_annual.loc[param_name].plot()
amazon_data.financial_statements_annual.loc[param_name].plot()
google_data.financial_statements_annual.loc[param_name].plot()
plt.plot()

#invert x-axis (2009 on the left, 2022 on the right)
ax = plt.gca()
ax.invert_xaxis()

plt.show()

In [53]:

#get all ticker-urls and put them in the database

from sqlite3 import DatabaseError, OperationalError


class getTickersData():

    def __init__(self):

        # self.tickers_data = self.download_tickers()
        # self.store_data_in_database(self.tickers_data, "Macrotrends_Tickers_URLs")

        self.tickers_data = self.get_from_database('tickers_data') #get tickers-data from database
        if self.tickers_data is None: #if tickers-data not found in database
            print(f'tickers-data not found in {database_file}')
            self.tickers_data = self.download_tickers()
            self.store_data_in_database(self.tickers_data, "tickers_data")

        else: #if tickers-data exists in database
            print(f'tickers-data found in {database_file}')
            print(self.tickers_data)

    def download_tickers(self):
        ticker_name = [] #store ticker-names f.e. 'APPL'
        company_name = [] #store company-names f.e. 'apple'
        tickers_url = 'https://www.macrotrends.net/assets/php/ticker_search_list.php?_=1664947632720' #donwnload ticker-page
        
        data_tickers = requests.get(tickers_url) #download tickers-page
        data_tickers = data_tickers.text #convert it to text
        data_tickers_json = json.loads(data_tickers) #convert the text to json-format

        for item in data_tickers_json: #loop over each item in json-format
            ticker_name_temp = item['s'].split('/')[0] #get ticker-name from current item
            company_name_temp = item['s'].split('/')[1] #get company-name from current item

            ticker_name.append(ticker_name_temp) #append ticker-name to list
            company_name.append(company_name_temp) #append company-name to list

        data = {'ticker_name':ticker_name, 'company_name':company_name} #store in dictionary
        output = pd.DataFrame(data)
        return output

    def store_data_in_database(self, data, name):
        conn = sqlite3.connect(database_file) #connect to sqlite3-database
        data.to_sql(name=name, con=conn, if_exists='replace',  index=False) #store dataframe to sqlite-database
        conn.close()
    
    def get_from_database(self, name):
        conn = sqlite3.connect(database_file)
        try:
            output = pd.read_sql_query(f"SELECT * from {name}", conn) #get the data from database and put it in pandas-dataframe
        except :
            print(f'ERROR: {name}, not found in {database_file}')
            conn.close()
            return None

        # output_index = output.set_index('index', inplace=False) #set the index-column as dataframe-index, needed because otherwise the dataframe index won't be parameter names but integers
        conn.close()
        # print(output)
        return output


        
output = getTickersData()
# print(output.tickers_data.iloc[1][0])
# ticker_num_rows= len(output.tickers_data)
# for i in range (150): #ticker_num_rows):
#     ticker_name = str(output.tickers_data.iloc[i][0]).split('/')[0]
#     print(ticker_name)
#     try:
#         ScrapeMacrotrend(ticker_name)
#     except:
#         print(f"can't scrape{ticker_name}")
#     time.sleep(0.3)



tickers-data found in company_database.db
     ticker_name                    company_name
0           AAPL                           apple
1           MSFT                       microsoft
2           GOOG                        alphabet
3          GOOGL                        alphabet
4           AMZN                          amazon
...          ...                             ...
6174         BUR                 burford-capital
6175        BCSF  bain-capital-specialty-finance
6176        SIOX                axovant-sciences
6177        ARLP      alliance-resource-partners
6178         ACH               aluminum-of-china

[6179 rows x 2 columns]


In [ ]:
output.tickers_data

In [12]:
foo = dict()
f'{foo=}'.split('=')[0]
'foo' 

'foo'